In [1]:
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, TimeDistributed, LSTM, Conv1D
from keras.layers import Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
num_set = 15               # number of training sets
num_train = 100            # number of samples in each training set
len_input = 2204           # number of spectrogram time steps in the input
num_freq = 201             # number of frequencies in the spectrogram
len_output = 550           # number of time steps in the output of our model

X = np.zeros((num_set*num_train, len_input, num_freq))
Y = np.zeros((num_set*num_train, len_output, 1))

for i in range(num_set):
    X[i*num_train:(i+1)*num_train,:,:] = np.load("./train/X_train" + str(i+1) + ".npy")
    Y[i*num_train:(i+1)*num_train,:,:] = np.load("./train/Y_train" + str(i+1) + ".npy")

In [3]:
def Model():

    model = Sequential()
    
    # layer 1: convolutional layer
    model.add(Conv1D(196, kernel_size=8, strides=4))             # CONV1D
    model.add(BatchNormalization(axis = -1))                     # Batch normalization
    model.add(Activation('relu'))                                # ReLu activation
    model.add(Dropout(0.20))                                    # dropout 

    # layer 2: LSTM layer
    model.add(LSTM(units = 64, return_sequences = True))         # LSTM
    model.add(Dropout(0.20))                                     # dropout
    model.add(BatchNormalization(axis = -1))                     # Batch normalization
    
    # layer 3: LSTM layer
    model.add(LSTM(units = 64, return_sequences = True))         # LSTM
    model.add(Dropout(0.20))                                     # dropout 
    model.add(BatchNormalization(axis = -1))                     # Batch normalization
    
    # layer 4: LSTM layer
    model.add(LSTM(units = 64, return_sequences = True))         # LSTM
    model.add(Dropout(0.20))                                     # dropout 
    model.add(BatchNormalization(axis = -1))                     # Batch normalization
    
    # layer 5: LSTM layer
    model.add(LSTM(units = 64, return_sequences = True))         # LSTM
    model.add(Dropout(0.20))                                     # dropout 
    model.add(BatchNormalization(axis = -1))                     # Batch normalization
    model.add(Dropout(0.20))                                     # dropout
    
    # layer 6: dense layer with sigmoid activation
    model.add(TimeDistributed(Dense(1, activation = "sigmoid"))) # dense
    
    return model 

In [4]:
model = Model()                                                                 # build a model
opt = Adam(lr=0.0008, beta_1=0.9, beta_2=0.999, decay=0.01)                     # set the optimizer
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])  # compile the model
model.fit(X, Y, batch_size = 128, epochs=30)                                    # fit the model






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
1500/1500 [==============================] - 48s 32ms/step - loss: 0.8075 - acc: 0.5335
Epoch 2/30
1500/1500 [==============================] - 40s 27ms/step - loss: 0.6940 - acc: 0.6078
Epoch 3/30
1500/1500 [==============================] - 40s 27ms/step - loss: 0.6057 - acc: 0.7061
Epoch 4/30
1500/1500 [==============================] - 40s 27ms/step - loss: 0.5173 - acc: 0.8029
Epoch 5/30
1500/1500 [==============================] - 40s 27ms/step - loss: 0.4211 - acc: 0.8906
Epoch 6/30
1500/1500 [==============================] - 40s 27ms/step - loss: 0.3313 - acc: 0.9453
Epoch 7/30
1500/1500 [==============================] - 40s 27ms/step - loss: 0.2721 - acc: 0.9707
Epoch 8/30
1500/1500 [==============================] - 40s 26ms/step - loss: 0.2334 - acc: 

In [5]:
model.save("model.h5")                          # save the model